In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.neighbors  import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [3]:
train = pd.read_csv("./data/train.csv")
display(train.head())
display(len(train.index))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


891

In [4]:
import pandas_profiling
pandas_profiling.ProfileReport(train)

Number of variables,12
Number of observations,891
Total Missing (%),8.1%
Total size in memory,83.6 KiB
Average record size in memory,96.1 B
Numeric,6
Categorical,4
Boolean,1
Date,0
Text (Unique),1
Rejected,0


In [3]:
display(train.describe(include='all'))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Panula, Mr. Jaako Arnold",male,NaN,NaN,NaN,347082,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [14]:
# Checking missing values
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [15]:
# Cleaning Data

# Columns that I do not think is necessary
train = train.drop(['Cabin','Name','PassengerId','Ticket'], axis=1)

# For missing values
train = train.dropna(subset=['Embarked'])
train["Age"].fillna(train.Age.median(), inplace=True)

# Casting
train = train.replace('male', 0).replace('female', 1)
train = train.replace("C",0).replace("Q",1).replace("S",2)

In [18]:
# Checking data
display(len(train.index))
display(train.isnull().sum())
display(train.dtypes)

889

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

Survived      int64
Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked      int64
dtype: object

In [32]:
# Cross Validate Models
X = train.drop('Survived', axis=1)
y = train.Survived
(train_X, test_X, train_y, test_y) = train_test_split(X, y, test_size = 0.0, random_state = 42)

models = {}
models["RandomForest"] = RandomForestClassifier()
models["GradientBoosting"] = GradientBoostingClassifier()
models["KNeighbors"] = KNeighborsClassifier()
models["DecisionTree"] = DecisionTreeClassifier()
models["SVM"] = SVC()

for name, model in models.items():
    kfold = KFold(n_splits=10, random_state=42, shuffle=True)
    scores = cross_val_score(model, train_X, train_y, cv=kfold, scoring="accuracy")
    display("model: %s, average score: %f" % (name, np.mean(scores)))


'model: RandomForest, average score: 0.800919'

'model: GradientBoosting, average score: 0.820072'

'model: KNeighbors, average score: 0.697395'

'model: DecisionTree, average score: 0.778447'

'model: SVM, average score: 0.698608'

In [34]:
# Builing model
model = GradientBoostingClassifier(random_state=42)
model.fit(X, y)
display('Train score: {}'.format(model.score(train_X, train_y)))


'Train score: 0.8953880764904387'

In [77]:
test = pd.read_csv("./data/test.csv")
display(test.isnull().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [78]:
# Preparing data for test data (To fit in train data)
test_X = test.drop(['Cabin','Name','PassengerId','Ticket'], axis=1)
test_X = test_X.replace('male', 0).replace('female', 1)
test_X = test_X.replace("C",0).replace("Q",1).replace("S",2)
test_X["Age"].fillna(test_X.Age.median(), inplace=True)
test_X["Fare"].fillna(test_X.Fare.median(), inplace=True)


In [79]:
# Predict
pred_y = model.predict(test_X)

# Formatting and output
import csv
with open("./result/predict_result_data.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["PassengerId", "Survived"])
    for pid, survived in zip(test['PassengerId'].astype(int), pred_y.astype(int)):
        writer.writerow([pid, survived])


In [80]:
pred = pd.read_csv("./result/predict_result_data.csv")
display(pred)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0
